In [1]:
import re
from lxml import etree
import urllib.request
import zipfile
from nltk.tokenize import word_tokenize, sent_tokenize

In [2]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
cd /content/drive/My\ Drive/Colab Notebooks/lecture202007

In [ ]:
urllib.request.urlretrieve("https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip", filename="ted_en-20160408.zip")
# 데이터 다운로드

with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
  target_text = etree.parse(z.open('ted_en-20160408.xml', 'r'))
  parse_text = '\n'.join(target_text.xpath('//content/text()'))
# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.

In [ ]:
ls -al

In [ ]:
parse_text[:500]

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
content_text = re.sub(r'\([^)]*\)', '', parse_text)
# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.

sent_text=sent_tokenize(content_text)
# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.

normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)
# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.

result = [word_tokenize(sentence) for sentence in normalized_text]
# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
print('총 샘플의 개수 : {}'.format(len(result)))

In [ ]:
for line in result[:3]: # 샘플 3개만 출력
    print(line)

# Word2Vec 훈련

In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=result, 
        size=100, window=5, 
        min_count=5, 
        workers=4, 
        sg=0)

*   size = 워드 벡터의 특징 값, 임베딩 벡터 차워
*   window = 컨텍스트 윈도우 크기
*   단어 최소 빈도 수 제한 
    (빈도가 적은 단어들은 학습하지 않는다.)
*   workers = 학습을 위한 프로세스 수
*   sg = 0은 CBOW, 1은 Skip-gram.



In [ ]:
model_result = model.wv.most_similar("man")
print(model_result)

# Word2Vec 모델 저장하고 로드하기

In [ ]:
from gensim.models import KeyedVectors
model.wv.save_word2vec_format('eng_w2v') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v") # 모델 로드

In [ ]:
ls -al

In [ ]:
model_result = loaded_model.most_similar("man")
print(model_result)

# 한국어 Word2Vec 만들기(네이버 영화 리뷰)

In [ ]:
import urllib.request
from konlpy.tag import Okt
from gensim.models.word2vec import Word2Vec
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
!pip install konlpy

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")

In [ ]:
cd /content/drive/My\ Drive/Colab Notebooks/lecture202007

In [ ]:
ls -al

In [ ]:
!tail -f ratings.txt

In [ ]:
train_data = pd.read_table('ratings.txt')

In [ ]:
train_data[:5]

In [ ]:
len(train_data)

In [ ]:
train_data.isnull().values.sum()

In [ ]:
train_data=train_data.dropna(how='any')

In [ ]:
train_data.isnull().values.sum()

In [ ]:
len(train_data)

In [ ]:
# 정규 표현식을 통한 한글 외 문자 제거
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

In [ ]:
train_data[:5]

In [ ]:
# 불용어 정의
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [ ]:
# 형태소 분석기 OKT를 사용한 토큰화 작업 (다소 시간 소요)
okt = Okt()
tokenized_kdata = []
for sentence in train_data['document']:
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    print("토큰화=>",temp_X)
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    tokenized_kdata.append(temp_X)
    print("불용어=>",temp_X)

In [ ]:
from gensim.models import KeyedVectors
model.wv.save_word2vec_format('eng_w2v_naver') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v_naver") # 모델 로드

In [ ]:
ls -al

In [ ]:
print(model.wv.most_similar("최민식"))

In [ ]:
model_result = loaded_model.most_similar("액션")
print(model_result)

In [ ]:
print(model.wv.most_similar("히어로"))

# 저장된 word2vec

In [ ]:
!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

In [ ]:
ls -al

In [ ]:
print(model.vectors.shape) # 모델의 크기 확인

In [ ]:
(3000000, 300)
# 모델의 크기는 3,000,000 x 300. 즉, 3백만 개의 단어와 각 단어의 차원은 300
# 파일의 크기가 3기가가 넘는 이유를 계산해보면 아래와 같다.
# 3 million words * 300 features * 4bytes/feature = ~3.35GB

In [ ]:
print (model.similarity('this', 'is')) # 두 단어의 유사도 계산하기
print (model.similarity('post', 'book'))
print (model.similarity('man', 'woman'))
print (model.similarity('dog', 'cat'))

In [ ]:
print(model['book']) # 단어 'book'의 벡터 출력

# 한국 word2vec


In [ ]:
ls -al

In [ ]:
!unzip ko.zip